# Quick Start

In [1]:
# Set project as root directory
import os

if os.getcwd().endswith("notebooks"):
    os.chdir("..")
print(os.getcwd())

/home/cmcouto-silva/Projects/sqldeps


## API

In [2]:
import sqldeps

print("sqldeps version:", sqldeps.__version__)

sqldeps version: 0.1.1


### Exploring SQL Extractor & SQLProfile

In [3]:
from sqldeps.llm_parsers import create_extractor

In [4]:
# `create_extractor` is a wrapper to instantiate the target SQL extractor
# Every extractor is derivd from the base class `BaseSQLExtractor`
# The default extractor is `LiteLLMExtractor`, which allows specifying models from multiple providers

create_extractor?

Signature:
create_extractor(
    framework: str = 'litellm',
    model: str | None = None,
    params: dict | None = None,
    prompt_path: pathlib._local.Path | None = None,
) -> sqldeps.llm_parsers.base.BaseSQLExtractor
Docstring:
Create an appropriate SQL extractor based on the specified framework.

Args:
    framework: The LLM framework to use ("litellm", "groq", "openai", or "deepseek")
        Note: Direct framework options are maintained for backward compatibility,
        but "litellm" is recommended as it provides integrations for all models
        from multiple providers
    model: The model name within the selected framework (uses default if None)
    params: Additional parameters to pass to the LLM API
    prompt_path: Path to a custom prompt YAML file

Returns:
    An instance of the appropriate SQL extractor

Raises:
    ValueError: If an unsupported framework is specified
File:      ~/Projects/sqldeps/sqldeps/llm_parsers/__init__.py
Type:      function

In [5]:
# Create extractor for the OpenAI GPT-4.1-mini model (expects env variable OPENAI_API_KEY to be set)
extractor = create_extractor(model="openai/gpt-4.1-mini")

In [6]:
# Extract dependencies and outputs from a SQL query
sql_query = """
WITH user_orders AS (
    SELECT o.user_id, COUNT(*) AS order_count
    FROM orders o
    JOIN users u ON o.user_id = u.id
    WHERE u.status = 'active'
    GROUP BY o.user_id
)

CREATE TABLE transactions.user_order_summary AS
SELECT * FROM user_orders;
"""
sql_profile = extractor.extract_from_query(sql_query)

# Print the extracted dependencies and outputs
print(sql_profile)

SQLProfile(dependencies={'orders': ['user_id'], 'users': ['id', 'status']}, outputs={'transactions.user_order_summary': ['*']})


In [7]:
# Extracted result object contains the dependencies and outputs
print("Dependencies:", sql_profile.dependencies)
print("Outputs:", sql_profile.outputs)

# Convert the result to different formats
print("As dictionary:", sql_profile.to_dict())

print("As DataFrame:")
print(sql_profile.to_dataframe())

Dependencies: {'orders': ['user_id'], 'users': ['id', 'status']}
Outputs: {'transactions.user_order_summary': ['*']}
As dictionary: {'dependencies': {'orders': ['user_id'], 'users': ['id', 'status']}, 'outputs': {'transactions.user_order_summary': ['*']}}
As DataFrame:
         type        schema               table   column
0  dependency          None              orders  user_id
1  dependency          None               users       id
2  dependency          None               users   status
3     outcome  transactions  user_order_summary        *


## Extracting SQL profile from a file

In [8]:
extractor.extract_from_file?

Signature: extractor.extract_from_file(file_path: str | pathlib._local.Path) -> sqldeps.models.SQLProfile
Docstring:
Extract dependencies from a SQL file.

Args:
    file_path: Path to SQL file

Returns:
    SQLProfile object containing dependencies and outputs

Raises:
    FileNotFoundError: If file does not exist
File:      ~/Projects/sqldeps/sqldeps/llm_parsers/base.py
Type:      method

In [9]:
extractor.extract_from_file("data/examples/example.sql")

SQLProfile(dependencies={'orders': ['user_id'], 'users': ['id', 'status']}, outputs={'order_summary': [], 'transactions.user_order_summary': ['*']})

## Extracting SQL profile from a folder

In [10]:
extractor.extract_from_folder?

Signature:
extractor.extract_from_folder(
    folder_path: str | pathlib._local.Path,
    recursive: bool = False,
    merge_sql_profiles: bool = False,
    valid_extensions: set[str] | None = None,
    n_workers: int = 1,
    rpm: int = 100,
    use_cache: bool = True,
    clear_cache: bool = False,
) -> sqldeps.models.SQLProfile | dict[str, sqldeps.models.SQLProfile]
Docstring:
Extract and merge dependencies from all SQL files in a folder.

Args:
    folder_path: Path to folder containing SQL files
    recursive: Whether to search recursively
    merge_sql_profiles: Whether to merge all results into a single SQLProfile
    valid_extensions: Set of valid file extensions to process
    n_workers: Number of worker processes for parallel execution
    rpm: Maximum requests per minute for API rate limiting
    use_cache: Whether to use cached results
    clear_cache: Whether to clear the cache after processing

Returns:
    SQLProfile object or dictionary mapping file paths to SQLProfile 

In [11]:
# Extract from folder with default parameters
extractor.extract_from_folder(folder_path = "data/examples/folders_with_sql_files")

2025-05-06 17:55:02.431 | INFO     | sqldeps.llm_parsers.base:_process_files_sequentially:180 - Cache usage: enabled
2025-05-06 17:55:02.432 | INFO     | sqldeps.llm_parsers.base:_process_files_sequentially:181 - Processing 2 SQL files sequentially with RPM: 100
Processing SQL files:   0%|          | 0/2 [00:00<?, ?it/s]2025-05-06 17:55:02.434 | INFO     | sqldeps.cache:load_from_cache:99 - Loading from cache: data/examples/folders_with_sql_files/example1.sql
2025-05-06 17:55:02.434 | INFO     | sqldeps.cache:load_from_cache:99 - Loading from cache: data/examples/folders_with_sql_files/example2.sql
Processing SQL files: 100%|██████████| 2/2 [00:00<00:00, 1973.33it/s]


{'data/examples/folders_with_sql_files/example1.sql': SQLProfile(dependencies={'users': ['id', 'name']}, outputs={}),
 'data/examples/folders_with_sql_files/example2.sql': SQLProfile(dependencies={'users': ['*']}, outputs={})}

In [12]:
# Extract from folder with default parameters
extractor.extract_from_folder(
    folder_path = "data/examples/folders_with_sql_files",
    merge_sql_profiles = True
)

2025-05-06 17:55:02.441 | INFO     | sqldeps.llm_parsers.base:_process_files_sequentially:180 - Cache usage: enabled
2025-05-06 17:55:02.442 | INFO     | sqldeps.llm_parsers.base:_process_files_sequentially:181 - Processing 2 SQL files sequentially with RPM: 100
Processing SQL files:   0%|          | 0/2 [00:00<?, ?it/s]2025-05-06 17:55:02.444 | INFO     | sqldeps.cache:load_from_cache:99 - Loading from cache: data/examples/folders_with_sql_files/example1.sql
2025-05-06 17:55:02.444 | INFO     | sqldeps.cache:load_from_cache:99 - Loading from cache: data/examples/folders_with_sql_files/example2.sql
Processing SQL files: 100%|██████████| 2/2 [00:00<00:00, 1301.77it/s]


SQLProfile(dependencies={'users': ['*']}, outputs={})

In [13]:
# Clean up cache
from sqldeps.cache import cleanup_cache

cleanup_cache()

2025-05-06 17:55:02.454 | INFO     | sqldeps.cache:cleanup_cache:126 - Removed cache directory: .sqldeps_cache


True

In [14]:
# Extract from folder recursively with multiprocessing
extractor.extract_from_folder(
    folder_path = "data/examples/folders_with_sql_files/",
    recursive = True,
    merge_sql_profiles = False,
    valid_extensions = None,
    n_workers = -1,   # use all available cores
    rpm = 100,        # Max 100 requests per minute
    use_cache = True,
    clear_cache = False,
)

2025-05-06 17:55:02.472 | INFO     | sqldeps.parallel:process_files_in_parallel:168 - Processing 5 SQL files
2025-05-06 17:55:02.473 | INFO     | sqldeps.parallel:process_files_in_parallel:169 - Using 20 workers with global rate limit of 100 requests per minute
2025-05-06 17:55:02.473 | INFO     | sqldeps.parallel:process_files_in_parallel:172 - Cache usage: enabled
2025-05-06 17:55:02.536 | DEBUG    | sqldeps.parallel:extract_with_rate_limit:85 - Extracting from file: data/examples/folders_with_sql_files/example2.sql
2025-05-06 17:55:02.537 | DEBUG    | sqldeps.parallel:extract_with_rate_limit:85 - Extracting from file: data/examples/folders_with_sql_files/example1.sql
2025-05-06 17:55:02.541 | DEBUG    | sqldeps.parallel:extract_with_rate_limit:85 - Extracting from file: data/examples/folders_with_sql_files/subfolder1/example3.sql
2025-05-06 17:55:02.542 | DEBUG    | sqldeps.parallel:extract_with_rate_limit:85 - Extracting from file: data/examples/folders_with_sql_files/subfolder2/su

{'data/examples/folders_with_sql_files/example2.sql': SQLProfile(dependencies={'users': ['*']}, outputs={}),
 'data/examples/folders_with_sql_files/subfolder1/example3.sql': SQLProfile(dependencies={'my_db.users': ['id', 'name'], 'orders': ['order_id', 'user_id']}, outputs={}),
 'data/examples/folders_with_sql_files/subfolder2/subfolder2_1/example5.sql': SQLProfile(dependencies={'orders': ['user_id'], 'users': ['id', 'name']}, outputs={}),
 'data/examples/folders_with_sql_files/subfolder2/subfolder2_1/example4.sql': SQLProfile(dependencies={'my_db.users': ['email', 'id', 'name', 'status'], 'orders': ['order_date', 'order_id', 'order_type', 'priority_level', 'shipping_status', 'total_amount', 'user_id']}, outputs={}),
 'data/examples/folders_with_sql_files/example1.sql': SQLProfile(dependencies={'users': ['id', 'name']}, outputs={})}

In [15]:
# Extract from folder recursively with multiprocessing with cached files
extractor.extract_from_folder(
    folder_path = "data/examples/folders_with_sql_files/",
    recursive = True,
    merge_sql_profiles = False,
    valid_extensions = None,
    n_workers = -1,   # use all available cores
    rpm = 100,        # Max 100 requests per minute
    use_cache = True,
    clear_cache = False,
)

2025-05-06 17:55:05.238 | INFO     | sqldeps.parallel:process_files_in_parallel:168 - Processing 5 SQL files
2025-05-06 17:55:05.238 | INFO     | sqldeps.parallel:process_files_in_parallel:169 - Using 20 workers with global rate limit of 100 requests per minute
2025-05-06 17:55:05.239 | INFO     | sqldeps.parallel:process_files_in_parallel:172 - Cache usage: enabled
2025-05-06 17:55:05.303 | INFO     | sqldeps.cache:load_from_cache:99 - Loading from cache: data/examples/folders_with_sql_files/example2.sql
2025-05-06 17:55:05.304 | INFO     | sqldeps.cache:load_from_cache:99 - Loading from cache: data/examples/folders_with_sql_files/subfolder1/example3.sql
2025-05-06 17:55:05.308 | INFO     | sqldeps.parallel:process_files_in_parallel:208 - Completed batch 3/5 with 1 results
2025-05-06 17:55:05.305 | INFO     | sqldeps.cache:load_from_cache:99 - Loading from cache: data/examples/folders_with_sql_files/example1.sql
2025-05-06 17:55:05.313 | INFO     | sqldeps.parallel:process_files_in_pa

{'data/examples/folders_with_sql_files/subfolder1/example3.sql': SQLProfile(dependencies={'my_db.users': ['id', 'name'], 'orders': ['order_id', 'user_id']}, outputs={}),
 'data/examples/folders_with_sql_files/example2.sql': SQLProfile(dependencies={'users': ['*']}, outputs={}),
 'data/examples/folders_with_sql_files/example1.sql': SQLProfile(dependencies={'users': ['id', 'name']}, outputs={}),
 'data/examples/folders_with_sql_files/subfolder2/subfolder2_1/example5.sql': SQLProfile(dependencies={'orders': ['user_id'], 'users': ['id', 'name']}, outputs={}),
 'data/examples/folders_with_sql_files/subfolder2/subfolder2_1/example4.sql': SQLProfile(dependencies={'my_db.users': ['email', 'id', 'name', 'status'], 'orders': ['order_date', 'order_id', 'order_type', 'priority_level', 'shipping_status', 'total_amount', 'user_id']}, outputs={})}

In [16]:
# Clean up cache again
cleanup_cache()

2025-05-06 17:55:05.359 | INFO     | sqldeps.cache:cleanup_cache:126 - Removed cache directory: .sqldeps_cache


True

In [17]:
# Extract from folder recursively with multiprocessing with cached files with max 3 requests per minute
extractor.extract_from_folder(
    folder_path = "data/examples/folders_with_sql_files/",
    recursive = True,
    merge_sql_profiles = False,
    valid_extensions = None,
    n_workers = -1,   # use all available cores
    rpm = 3,        # Max 3 requests per minute
    use_cache = True,
    clear_cache = False,
)

2025-05-06 17:55:05.369 | INFO     | sqldeps.parallel:process_files_in_parallel:168 - Processing 5 SQL files
2025-05-06 17:55:05.370 | INFO     | sqldeps.parallel:process_files_in_parallel:169 - Using 20 workers with global rate limit of 3 requests per minute
2025-05-06 17:55:05.370 | INFO     | sqldeps.parallel:process_files_in_parallel:172 - Cache usage: enabled
2025-05-06 17:55:05.449 | DEBUG    | sqldeps.parallel:extract_with_rate_limit:85 - Extracting from file: data/examples/folders_with_sql_files/example2.sql
2025-05-06 17:55:05.452 | DEBUG    | sqldeps.parallel:extract_with_rate_limit:85 - Extracting from file: data/examples/folders_with_sql_files/example1.sql
2025-05-06 17:55:05.453 | DEBUG    | sqldeps.parallel:extract_with_rate_limit:85 - Extracting from file: data/examples/folders_with_sql_files/subfolder2/subfolder2_1/example4.sql
2025-05-06 17:55:05.455 | DEBUG    | sqldeps.rate_limiter:wait_if_needed:110 - Rate limit reached. Waiting 59.99 seconds
2025-05-06 17:55:06.451

{'data/examples/folders_with_sql_files/example1.sql': SQLProfile(dependencies={'users': ['id', 'name']}, outputs={}),
 'data/examples/folders_with_sql_files/subfolder2/subfolder2_1/example4.sql': SQLProfile(dependencies={'my_db.users': ['email', 'id', 'name', 'status'], 'orders': ['order_date', 'order_id', 'order_type', 'priority_level', 'shipping_status', 'total_amount', 'user_id']}, outputs={}),
 'data/examples/folders_with_sql_files/example2.sql': SQLProfile(dependencies={'users': ['*']}, outputs={}),
 'data/examples/folders_with_sql_files/subfolder2/subfolder2_1/example5.sql': SQLProfile(dependencies={'orders': ['user_id'], 'users': ['id', 'name']}, outputs={}),
 'data/examples/folders_with_sql_files/subfolder1/example3.sql': SQLProfile(dependencies={'my_db.users': ['id', 'name'], 'orders': ['order_id', 'user_id']}, outputs={})}

### SQL Profile visualization

In [18]:
from sqldeps.visualization import visualize_sql_dependencies

In [19]:
sql_profiles = extractor.extract_from_folder("data/examples/folders_with_sql_files/", recursive = True)

# Visualize the SQL dependencies
fig = visualize_sql_dependencies(sql_profiles)
fig.update_layout(height=800)

2025-05-06 17:56:07.081 | INFO     | sqldeps.llm_parsers.base:_process_files_sequentially:180 - Cache usage: enabled
2025-05-06 17:56:07.082 | INFO     | sqldeps.llm_parsers.base:_process_files_sequentially:181 - Processing 5 SQL files sequentially with RPM: 100
Processing SQL files:   0%|          | 0/5 [00:00<?, ?it/s]2025-05-06 17:56:07.084 | INFO     | sqldeps.cache:load_from_cache:99 - Loading from cache: data/examples/folders_with_sql_files/example1.sql
2025-05-06 17:56:07.084 | INFO     | sqldeps.cache:load_from_cache:99 - Loading from cache: data/examples/folders_with_sql_files/example2.sql
2025-05-06 17:56:07.084 | INFO     | sqldeps.cache:load_from_cache:99 - Loading from cache: data/examples/folders_with_sql_files/subfolder1/example3.sql
2025-05-06 17:56:07.085 | INFO     | sqldeps.cache:load_from_cache:99 - Loading from cache: data/examples/folders_with_sql_files/subfolder2/subfolder2_1/example5.sql
2025-05-06 17:56:07.086 | INFO     | sqldeps.cache:load_from_cache:99 - Loa

## CLI

In [20]:
%%bash

sqldeps --version

SQLDeps version: 0.1.1


In [21]:
%%bash

sqldeps --help

                                                                                
 Usage: sqldeps [OPTIONS] COMMAND [ARGS]...                                     
                                                                                
 SQL Dependency Extractor - Analyze SQL files to extract table and column       
 dependencies                                                                   
                                                                                
                                                                                
╭─ Options ─────────────────────��──────────────────────────────────────────────╮
│ --version                     Show the version and exit.                     │
│ --install-completion          Install completion for the current shell.      │
│ --show-completion             Show completion for the current shell, to copy │
│                               it or customize the installation.              │
│ --help                   

In [22]:
%%bash

sqldeps extract --help

                                                                                
 Usage: sqldeps extract [OPTIONS] FPATH                                         
                                                                                
 Extract SQL dependencies from file or folder.                                  
                                                                                
 This tool analyzes SQL files to identify table and column dependencies,        
 optionally validating them against a real database schema.                     
                                                                                
╭─ Arguments ───────────────────────────────────────────────���──────────────────╮
│ *    fpath      PATH  SQL file or directory path [default: None] [required]  │
╰────────────────────────────────────────────────────────────���─────────────────╯
╭─ Options ─────────────────────��──────────────────────────────────────────────╮
│ --framework          

In [23]:
%%bash

# Extract from file
sqldeps extract data/examples/example.sql

2025-05-06 17:56:14.368 | INFO     | sqldeps.cli:extract_dependencies:74 - Extracting dependencies from file: /home/cmcouto-silva/Projects/sqldeps/data/examples/example.sql
2025-05-06 17:56:15.077 | SUCCESS  | sqldeps.cli:save_output:183 - Saved to JSON: dependencies.json


In [24]:
%%bash

# Extract from folder resursively & return CSV (accepts JSON and CSV as output formats)
sqldeps extract data/examples/folders_with_sql_files --recursive -o project_sql_profiles.csv

2025-05-06 17:56:16.979 | INFO     | sqldeps.cli:extract_dependencies:74 - Extracting dependencies from folder: /home/cmcouto-silva/Projects/sqldeps/data/examples/folders_with_sql_files
2025-05-06 17:56:16.979 | INFO     | sqldeps.llm_parsers.base:_process_files_sequentially:180 - Cache usage: enabled
2025-05-06 17:56:16.979 | INFO     | sqldeps.llm_parsers.base:_process_files_sequentially:181 - Processing 5 SQL files sequentially with RPM: 100
Processing SQL files:   0%|          | 0/5 [00:00<?, ?it/s]2025-05-06 17:56:16.980 | INFO     | sqldeps.cache:load_from_cache:99 - Loading from cache: /home/cmcouto-silva/Projects/sqldeps/data/examples/folders_with_sql_files/example1.sql
2025-05-06 17:56:16.980 | INFO     | sqldeps.cache:load_from_cache:99 - Loading from cache: /home/cmcouto-silva/Projects/sqldeps/data/examples/folders_with_sql_files/example2.sql
2025-05-06 17:56:16.980 | INFO     | sqldeps.cache:load_from_cache:99 - Loading from cache: /home/cmcouto-silva/Projects/sqldeps/data/

In [25]:
%%bash

# Clear cache
sqldeps cache clear

2025-05-06 17:56:18.748 | INFO     | sqldeps.cli:cache_clear:379 - Clearing SQLDeps cache...
2025-05-06 17:56:18.748 | INFO     | sqldeps.cache:cleanup_cache:126 - Removed cache directory: .sqldeps_cache
2025-05-06 17:56:18.748 | SUCCESS  | sqldeps.cli:cache_clear:382 - Cache cleared successfully
